In [57]:
import os
import json
import random

In [58]:
benchmarks = ["HumanEval"]
models = ["llama3", "llama3.1", "mistral-nemo", "qwen2.5-coder"]

def tie_break(data):
    ties = [i for i in data if i[1][0] == data[0][1][0]]
    return random.choice(ties)


def vote(mode, benchmark, model):
    with open(f"./preprocessed_results/{mode}/{benchmark}_{model}.json", "r") as f:
        data =json.load(f)
    result = []

    for problem_id, samples in data.items():
        sample_dict = dict()
        for sample in samples:
            if sample[0] not in sample_dict:
                sample_dict[sample[0]] = [1, sample[1]["result"]]
            elif sample[0] in sample_dict:
                sample_dict[sample[0]][0] += 1
        major = sorted(sample_dict.items(), key=lambda x:x[1][0], reverse=True)
        if "passed" in tie_break(major)[1][1]:
            result.append(True)
        else:
            result.append(False)
        
    return result

## Augmented

In [59]:
for bm in benchmarks:
    print(bm)
    for model in models:
        result = vote("augmented", bm, model + "_augmented")
        print(f"{model}: {result.count(True)}/{len(result)}")

HumanEval
llama3: 50/164
llama3.1: 82/164
mistral-nemo: 53/164
qwen2.5-coder: 109/164


## AST-only

In [60]:
for bm in benchmarks:
    print(bm)
    for model in models:
        result = vote("ast_only", bm, model)
        print(f"{model}: {result.count(True)}/{len(result)}")

HumanEval
llama3: 61/164
llama3.1: 89/164
mistral-nemo: 58/164
qwen2.5-coder: 107/164


## Variable Unified

In [61]:
for bm in benchmarks:
    print(bm)
    for model in models:
        result = vote("var_unif", bm, model)
        print(f"{model}: {result.count(True)}/{len(result)}")

HumanEval
llama3: 58/164
llama3.1: 90/164
mistral-nemo: 60/164
qwen2.5-coder: 109/164
